In [2]:
import json # read from json file
import re
import sklearn
from sklearn.utils import shuffle
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
import scipy.stats
import eli5

#with open('data.jsonl') as jsonl_file: # not using the full (unannotated) dataset in the below
#    lines = jsonl_file.readlines()
#jsonS = [json.loads(line) for line in lines]
#print(len(jsonS))

In [3]:
### RUN
with open('annotations2.jsonl') as jsonl_file:
    # note: after running data-preprocessing.ipynb this file already has token-level labels
    lines = jsonl_file.readlines()
annot = [json.loads(line) for line in lines]
#print(annot[0])

Chose whether or not to add leading "B-" / "I-" to class labels

In [3]:
########## ONLY RUN IF WE WANT TO ADD LEADING "B-" / "I-" TO CLASS LABEL
# now use above code and loop through all items of annot list:
# addLeading=1 for "Yes" (i.e. add leading "B-","I-" to annot); 0 for "No" (i.e. add labels to annot simply as they are)
addLeading = 1

if addLeading == 1:
    for j in range(0,len(annot)):
        a = annot[j]
        # select list of dict of tokens w/ annnotations and add column w/ no. of words to each dict:
        b = a['spans']
        # add noWords to b dict. note: b is list of dicts w/ annotations; tokens not on this list don't have annotations
        if b!=[]: #i.e. only try to add annotations to tokens if there are annotations to begin with
            #print(b)
            for i in range(0,len(annot[j]['tokens'])):
                    # now break-up label into 1st occurrence (leading "B-") and subsequent occurrences (leading "I-") (only for non "O"'s)
                    if annot[j]['tokens'][i]['label'] != "O":
                        if i==0:
                            annot[j]['tokens'][i]['label'] = "B-" + annot[j]['tokens'][i]['label']
                        else: 
                            if annot[j]['tokens'][i]['label'] == annot[j]['tokens'][i-1]['label'][2:]: # need to remove the leading "B-" that we had already been added to c[i-1]
                                annot[j]['tokens'][i]['label'] = "I-" + annot[j]['tokens'][i]['label']
                            else:
                                annot[j]['tokens'][i]['label'] = "B-" + annot[j]['tokens'][i]['label'] 

In [4]:
### RUN
# now convert annotation tokens into list (sentences) of lists (tokens) format for sklearn_crfsuite.CRF
train_sents=[] 
for j in range(0,len(annot)):
    a = annot[j]['tokens']
    train_sentence = []
    for i in range(0,len(a)):
        if 'label' in a[i]: # only add element if this sample sentence has been labelled 
            token_element = (a[i]['text'],a[i]['label'])
            train_sentence.append(token_element)
    train_sents.append(train_sentence)

In [6]:
### RUN
import pandas as pd
df = pd.read_csv('townsVoralberg.csv',names=['towns'])  

In [7]:
### RUN
lCase = lambda a: str(a).lower()

def word2features(sent, i):
    word = sent[i][0]
    word_l = word.lower()
    towns = map(lCase,df['towns'])
    #postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word_l,
        'word[-3:]': word[-3:], # 3 char ending of word (suffix)
        'word[-2:]': word[-2:], # 2 char ending of word (suffix)
        'word.isupper()': word.isupper(), # all letters are caps
        'word.istitle()': word.istitle(), # 1st letter is a capital letter
        'word.isdigit()': word.isdigit(), # is digit?
        'word.isPrice()': bool(re.match(r'(\d\.?)?(\d{3}\.?\d{3})',word)),
        'word.isTown()': word_l in towns,
    }
    if i > 0:
        word1 = sent[i-1][0] # enriches feature function w/ immediately preceding word
        word1_l = word1.lower()
        #postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1_l,
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:word.isPrice()': bool(re.match(r'(\d\.?)?(\d{3}\.?\d{3})',word1)),
            '-1.word.isTown()': word1_l in towns,
        })
    else:
        features['BOS'] = True # adds the start token

    if i < len(sent)-1:
        word1 = sent[i+1][0] # ... as well as w/ immediate sucessor word 
        word1_l = word1.lower()
        #postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1_l,
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1word.isPrice()': bool(re.match(r'(\d\.?)?(\d{3}\.?\d{3})',word1)),
            '+1.word.isTown()': word1_l in towns,
        })
    else:
        features['EOS'] = True # adds the end token

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
   # return [label for token, postag, label in sent]
    return [label for token, label in sent]

def sent2tokens(sent):
   # return [token for token, postag, label in sent]
    return [token for token, label in sent]

In [8]:
X = [sent2features(s) for s in train_sents] # both are 140 x # of tokens in each example
y = [sent2labels(s) for s in train_sents]

In [9]:
train_ratio = 0.75
train_test_split = round(0.75*len(X) - 0.5) # -0.5 => floor
idx = [i for i in range(0,len(X))]
idx_shuffle = shuffle(idx,random_state=0)
X_shuffle, y_shuffle = [X[auxIdx] for auxIdx in idx_shuffle], [y[auxIdx] for auxIdx in idx_shuffle]
X_train, X_test, y_train, y_test = X_shuffle[:train_test_split], X_shuffle[train_test_split:], y_shuffle[:train_test_split], y_shuffle[train_test_split:]

In [10]:
%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

Wall time: 0 ns


C:\Users\vasco\anaconda3\envs\xcs224u\lib\site-packages\sklearn\base.py:209: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100)

In [12]:
labels = list(crf.classes_)
y_pred = crf.predict(X_test)
print(metrics.flat_f1_score(y_test, y_pred,average='macro', labels=labels))
print(metrics.sequence_accuracy_score(y_test, y_pred))

0.8859668160921159
0.3055555555555556


In [13]:
# group B and I results
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

                     precision    recall  f1-score   support

                  O      0.971     0.995     0.983      1472
            KAEUFER      0.513     0.606     0.556        33
DATUM_VERBUECHERUNG      1.000     0.983     0.991        58
      DATUM_VERTRAG      1.000     0.922     0.959        64
         VERKAEUFER      0.841     0.552     0.667        67
   TERRASSENGROESSE      0.818     1.000     0.900         9
        GESAMTPREIS      1.000     0.793     0.885        29
            FLAECHE      1.000     0.974     0.987        38
           IMMO_TYP      0.889     0.800     0.842        50
            QMPREIS      1.000     1.000     1.000        23
                ORT      0.984     1.000     0.992        61
            STRASSE      0.902     0.841     0.871        44

           accuracy                          0.958      1948
          macro avg      0.910     0.872     0.886      1948
       weighted avg      0.958     0.958     0.957      1948



C:\Users\vasco\anaconda3\envs\xcs224u\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass labels=['O', 'KAEUFER', 'DATUM_VERBUECHERUNG', 'DATUM_VERTRAG', 'VERKAEUFER', 'TERRASSENGROESSE', 'GESAMTPREIS', 'FLAECHE', 'IMMO_TYP', 'QMPREIS', 'ORT', 'STRASSE'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


In [14]:
####### RUN THIS ONE
eli5.show_weights(crf, top=30)

In [15]:
from collections import Counter

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))

print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

Top likely transitions:
DATUM_VERTRAG -> DATUM_VERTRAG 5.001647
KAEUFER -> KAEUFER 4.633315
VERKAEUFER -> VERKAEUFER 4.535609
STRASSE -> STRASSE 4.157877
IMMO_TYP -> IMMO_TYP 4.003745
DATUM_VERBUECHERUNG -> DATUM_VERBUECHERUNG 3.919460
O      -> O       2.420556
ORT    -> O       1.460406
GESAMTPREIS -> GESAMTPREIS 1.275251
FLAECHE -> FLAECHE 1.232595
QMPREIS -> QMPREIS 1.143765
TERRASSENGROESSE -> O       1.095506
O      -> ORT     1.081547
FLAECHE -> O       1.005788
O      -> FLAECHE 0.986099
GESAMTPREIS -> O       0.979653
QMPREIS -> O       0.935153
O      -> GESAMTPREIS 0.614090
O      -> TERRASSENGROESSE 0.521701
STRASSE -> O       0.312111

Top unlikely transitions:
VERKAEUFER -> IMMO_TYP -0.269014
IMMO_TYP -> DATUM_VERBUECHERUNG -0.284437
DATUM_VERTRAG -> KAEUFER -0.298023
IMMO_TYP -> KAEUFER -0.314359
IMMO_TYP -> VERKAEUFER -0.383739
ORT    -> IMMO_TYP -0.385320
VERKAEUFER -> O       -0.408767
ORT    -> KAEUFER -0.423716
O      -> DATUM_VERTRAG -0.483720
IMMO_TYP -> TERRASSEN